In [8]:
import cv2
import numpy as np

# Cargar imagen de referencia (la señal que quieres detectar)
img_ref = cv2.imread('alto.jpg', cv2.IMREAD_GRAYSCALE)
if img_ref is None:
    print("No se encontró la imagen de referencia.")
    exit()

# Crear detector ORB
orb = cv2.ORB_create(nfeatures=2000)

# Detectar keypoints y descriptores en la imagen de referencia
kp_ref, des_ref = orb.detectAndCompute(img_ref, None)

# Crear matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Iniciar cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar en la imagen de la cámara
    kp_frame, des_frame = orb.detectAndCompute(gray, None)
    if des_frame is None or len(kp_frame) == 0:
        continue

    # Hacer matching
    matches = bf.match(des_ref, des_frame)
    good_matches = sorted(matches, key=lambda x: x.distance)[:20]

    if len(good_matches) >= 10:
        # Obtener puntos de correspondencia
        pts_ref = np.float32([kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        pts_frame = np.float32([kp_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Calcular homografía
        H, mask = cv2.findHomography(pts_ref, pts_frame, cv2.RANSAC, 5.0)

        # Si la homografía es válida, aplicar la transformación
        if H is not None:
            h, w = img_ref.shape
            pts_ref_corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            pts_frame_corners = cv2.perspectiveTransform(pts_ref_corners, H)

            # Dibujar el polígono de la señal detectada
            frame = cv2.polylines(frame, [np.int32(pts_frame_corners)], True, (0, 255, 0), 3)

            # Cálculo para centrar
            center_frame_x = frame.shape[1] / 2
            cx = np.mean([pt[0][0] for pt in pts_frame_corners])

            # Mostrar si está centrada
            if abs(cx - center_frame_x) < 30:
                cv2.putText(frame, "SENAL CENTRADA", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                if cx < center_frame_x:
                    texto = "GIRAR A LA IZQUIERDA"
                else:
                    texto = "GIRAR A LA DERECHA"
                cv2.putText(frame, texto, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Mostrar resultado
    cv2.imshow("Deteccion", frame)

    # Salir con 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
